# MLB Roster Optimization for 2025 Season

This notebook uses Mixed Integer Programming (MIP) to build an optimal MLB roster that:
- **Maximizes total projected WAR (xWAR)** for 2025
- **Stays within a salary budget**
- **Satisfies positional requirements**

## Prerequisites

Before running this notebook, you must:

1. **Train the models** by running these notebooks (in order):
   - `hitter_war_t_to_tp1_multiyear_backfill.ipynb` → saves `models/hitter_war_model.joblib`
   - `pitcher_war_t_to_tp1_multiyear_optimized.ipynb` → saves `models/pitcher_war_model.joblib`
   - `aav_regression.ipynb` → saves `models/hitter_salary_model.joblib` & `models/pitcher_salary_model.joblib`

2. **Prepare the player data** by running:
   ```bash
   python prepare_roster_data.py
   ```
   This generates `players.csv` with predicted 2025 xWAR and expected costs.

3. **Julia Requirements**: Julia with packages `CSV`, `DataFrames`, `JuMP`, and `Gurobi`


## 1) Load Julia Packages


In [11]:
using CSV
using DataFrames
using JuMP
using Gurobi


## 2) Load Player Data

Load the `players.csv` file generated by `prepare_roster_data.py`. This file contains:
- `player_id`: Unique MLB player ID
- `name`: Player name
- `position`: Player's primary position (C, 1B, 2B, 3B, SS, LF, CF, RF, DH, SP, RP)
- `xwar`: Predicted WAR/162 for 2025 season
- `cost`: Expected salary in millions of dollars


In [12]:
# Load player data generated by prepare_roster_data.py
players_df = CSV.read("players.csv", DataFrame)

println("Loaded $(nrow(players_df)) players from players.csv")

# Expect columns: player_id, name, position, xwar, cost
required_cols = ["player_id", "name", "position", "xwar", "cost"]
@assert all(c -> c ∈ names(players_df), required_cols) "Missing required columns in players.csv"

# Filter out any players with negative xWAR (not valuable)
players_df = filter(row -> row.xwar > 0, players_df)
println("After filtering to positive xWAR: $(nrow(players_df)) players")

player_ids = unique(players_df.player_id)
positions  = unique(players_df.position)

# Maps for convenience
player_name = Dict(p => players_df.name[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)
player_pos  = Dict(p => players_df.position[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)
xwar        = Dict(p => players_df.xwar[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)
cost        = Dict(p => players_df.cost[findfirst(==(p), players_df.player_id)]
                   for p in player_ids)

# Players by position
players_by_pos = Dict{String, Vector{Int}}()
for r in positions
    players_by_pos[r] = [p for p in player_ids if player_pos[p] == r]
end

# Print position summary
println("\nPlayers by position:")
for pos in sort(collect(keys(players_by_pos)))
    println("  $pos: $(length(players_by_pos[pos])) players")
end

# Show top 5 players by xWAR
println("\nTop 10 players by xWAR:")
top_players = sort(player_ids, by=p -> -xwar[p])[1:min(10, length(player_ids))]
for p in top_players
    println("  $(rpad(player_name[p], 25)) $(rpad(player_pos[p], 4)) xWAR=$(round(xwar[p], digits=2))  \$$(round(cost[p], digits=2))M")
end


Loaded 1462 players from players.csv
After filtering to positive xWAR: 993 players

Players by position:
  1B: 32 players
  2B: 46 players
  3B: 48 players
  C: 59 players
  CF: 62 players
  DH: 10 players
  LF: 39 players
  RF: 57 players
  RP: 335 players
  SP: 207 players
  SS: 83 players

Top 10 players by xWAR:
  Tyler Anderson            SP   xWAR=10.0  $14.1M
  Spencer Arrighetti        SP   xWAR=10.0  $0.66M
  Brayan Bello              SP   xWAR=10.0  $0.8M
  Jos\xc3\xa9 Berr\xc3\xados SP   xWAR=10.0  $0.75M
  Tanner Bibee              SP   xWAR=10.0  $0.76M
  Ronel Blanco              SP   xWAR=10.0  $5.62M
  Taj Bradley               SP   xWAR=10.0  $0.64M
  Hunter Brown              SP   xWAR=10.0  $0.78M
  Corbin Burnes             SP   xWAR=10.0  $5.9M
  Luis Castillo             SP   xWAR=10.0  $8.68M


## 2) Model Parameters


In [13]:
# Team budget (in millions of $)
budget = 100.0  # Competitive team budget (~$250M)

# Max roster size
roster_max = 26

# Positional min/max counts for a 26-man MLB roster
# Hitters: 13 (2C, 1 1B, 1 2B, 1 SS, 1 3B, 3 OF, 1 DH, 3 bench)
# Pitchers: 13 (5 SP, 8 RP)
pos_min = Dict(
    "C"  => 2,   # Need 2 catchers
    "1B" => 1,
    "2B" => 1,
    "3B" => 1,
    "SS" => 1,
    "LF" => 1,
    "CF" => 1,
    "RF" => 1,
    "DH" => 0,   # DH optional (can use other players)
    "SP" => 5,   # 5-man rotation
    "RP" => 6,   # At least 6 relievers
)

pos_max = Dict(
    "C"  => 3,
    "1B" => 2,
    "2B" => 2,
    "3B" => 2,
    "SS" => 2,
    "LF" => 3,
    "CF" => 3,
    "RF" => 3,
    "DH" => 2,
    "SP" => 6,
    "RP" => 9,
)

# Ensure every position in the data has bounds
for r in positions
    if !haskey(pos_min, r)
        @warn "No pos_min for $r, setting to 0"
        pos_min[r] = 0
    end
    if !haskey(pos_max, r)
        @warn "No pos_max for $r, setting to roster_max"
        pos_max[r] = roster_max
    end
end

println("Budget: \$$budget M")
println("Roster size: $roster_max players")
println("\nPositions available: ", join(positions, ", "))


Budget: $100.0 M
Roster size: 26 players

Positions available: SS, LF, RF, CF, C, 2B, 1B, DH, 3B, SP, RP


## 3) Build Model (Layer 1 + Positional Constraints)


In [14]:
env = Gurobi.Env()
model = Model(() -> Gurobi.Optimizer(env))
set_silent(model)  # comment this out if you want Gurobi logs

# Decision: x[p] = 1 if we roster/sign player p
@variable(model, x[p in player_ids], Bin)

# Objective: maximize total WAR
@objective(model, Max, sum(x[p] * xwar[p] for p in player_ids))

# Budget
@constraint(model, sum(x[p] * cost[p] for p in player_ids) <= budget)

# Roster size
@constraint(model, sum(x[p] for p in player_ids) <= roster_max)

# Positional min/max counts
for r in positions
    P_r = players_by_pos[r]
    if isempty(P_r)
        continue
    end
    @constraint(model, sum(x[p] for p in P_r) >= pos_min[r])
    @constraint(model, sum(x[p] for p in P_r) <= pos_max[r])
end


Set parameter Username
Set parameter LicenseID to value 2703863
Academic license - for non-commercial use only - expires 2026-09-05


In [15]:
optimize!(model)

println("Solver Status: ", termination_status(model))

if termination_status(model) != MOI.OPTIMAL &&
   termination_status(model) != MOI.LOCALLY_OPTIMAL
    error("Solver did not find an optimal solution.")
end

total_war  = objective_value(model)
total_cost = sum(cost[p] * value(x[p]) for p in player_ids)

println("\n" * "="^70)
println("   OPTIMAL 2025 ROSTER")
println("="^70)
println("\nTotal Projected xWAR: ", round(total_war, digits=2))
println("Total Payroll: \$", round(total_cost, digits=2), "M")
println("Budget Remaining: \$", round(budget - total_cost, digits=2), "M")

# Group selected players by position
selected = [p for p in player_ids if value(x[p]) > 0.5]
position_groups = Dict{String, Vector{Int}}()
for p in selected
    pos = player_pos[p]
    if !haskey(position_groups, pos)
        position_groups[pos] = []
    end
    push!(position_groups[pos], p)
end

# Print by position group
println("\n" * "-"^70)
println("HITTERS")
println("-"^70)
println(rpad("Player", 25), rpad("Pos", 5), rpad("xWAR", 10), "Cost")
println("-"^70)

hitter_positions = ["C", "1B", "2B", "SS", "3B", "LF", "CF", "RF", "DH"]
hitter_war = 0.0
hitter_cost = 0.0
for pos in hitter_positions
    if haskey(position_groups, pos)
        for p in sort(position_groups[pos], by=p -> -xwar[p])
            println(rpad(player_name[p], 25), rpad(pos, 5), rpad(round(xwar[p], digits=2), 10), "\$$(round(cost[p], digits=2))M")
            hitter_war += xwar[p]
            hitter_cost += cost[p]
        end
    end
end
println("-"^70)
println(rpad("Hitters Total", 25), rpad("", 5), rpad(round(hitter_war, digits=2), 10), "\$$(round(hitter_cost, digits=2))M")

println("\n" * "-"^70)
println("PITCHERS")
println("-"^70)
println(rpad("Player", 25), rpad("Pos", 5), rpad("xWAR", 10), "Cost")
println("-"^70)

pitcher_positions = ["SP", "RP"]
pitcher_war = 0.0
pitcher_cost = 0.0
for pos in pitcher_positions
    if haskey(position_groups, pos)
        for p in sort(position_groups[pos], by=p -> -xwar[p])
            println(rpad(player_name[p], 25), rpad(pos, 5), rpad(round(xwar[p], digits=2), 10), "\$$(round(cost[p], digits=2))M")
            pitcher_war += xwar[p]
            pitcher_cost += cost[p]
        end
    end
end
println("-"^70)
println(rpad("Pitchers Total", 25), rpad("", 5), rpad(round(pitcher_war, digits=2), 10), "\$$(round(pitcher_cost, digits=2))M")

println("\n" * "="^70)
println("ROSTER SUMMARY")
println("="^70)
println("Total Players: $(length(selected))")
println("Hitters: $(sum(length(get(position_groups, p, [])) for p in hitter_positions))")
println("Pitchers: $(sum(length(get(position_groups, p, [])) for p in pitcher_positions))")
println("\nProjected Team xWAR: $(round(total_war, digits=2))")
println("Total Payroll: \$$(round(total_cost, digits=2))M")
println("="^70)


Solver Status: OPTIMAL

   OPTIMAL 2025 ROSTER

Total Projected xWAR: 187.15
Total Payroll: $99.83M
Budget Remaining: $0.17M

----------------------------------------------------------------------
HITTERS
----------------------------------------------------------------------
Player                   Pos  xWAR      Cost
----------------------------------------------------------------------
William Contreras        C    4.55      $4.34M
Cal Raleigh              C    3.61      $2.73M
Vladimir Guerrero Jr.    1B   5.95      $5.35M
Andr\xc3\xa9s Gim\xc3\xa9nez2B   3.33      $0.86M
Bobby Witt Jr.           SS   9.01      $8.38M
Gunnar Henderson         SS   6.12      $1.48M
Jos\xc3\xa9 Ram\xc3\xadrez3B   5.35      $15.28M
Juan Soto                LF   6.72      $8.21M
Aaron Judge              CF   7.61      $26.0M
Corbin Carroll           RF   4.24      $0.77M
Yordan Alvarez           DH   5.59      $9.04M
----------------------------------------------------------------------
Hitters Total  